## ticker -> id

In [3]:
import requests
url = "https://www.athexgroup.gr/web/guest/companies-map"
resp = requests.get(url)


In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(resp.text)

In [5]:
dd = {}
table = soup.select("#companies-page")[0]
for div in table.find_all("div", recursive=False):
    a = div.find("a")
    dd[a.text.strip()] = a["href"].split("/")[-1]

In [6]:
import os, json
with open(os.path.join("/Users/a2/code/fin/trade/static/tickers", "AT_ticker_id.json"), "w") as f:
    json.dump(dd, f)

## ticker -> webseite

In [26]:
import os, json
import time, re
import requests
from bs4 import BeautifulSoup


url_template = "https://www.athexgroup.gr/company-profile/-/select-company/"
with open(os.path.join("/Users/a2/code/fin/trade/static/tickers", "AT_ticker_id.json"), "r") as f:
    dd = json.load(f)

n_val = {1:"activity",
         6:"webpage",
         10:"staffnumber",
         11:"staffnumber",
         15:"listed_on",
         }

p = "/Users/a2/code/fin/trade/data/fundamentals/company/info"
for ticker in dd.keys():
    if os.path.isfile(os.path.join(p, ticker+".json")):
        continue
    resp = requests.get(url_template+dd[ticker])
    print(url_template+dd[ticker])

    time.sleep(5)
    sp = BeautifulSoup(resp.text)
    temp = {}
    table = sp.select("#list-company-prof > table")[0]
    for n, tr in enumerate(table.find_all("tr")):
        tds = tr.find_all("td")
        if tds[0].text == "Sector / Subsector":
            print(tds)
            ll = tds[-1].text.split("/")
            try:
                temp["subsector"] = ll[1].replace(" ", " ").strip()
                temp["subsector"] = re.match(r"(.*) \(.+\)$", temp["subsector"]).group(1)
                temp["sector"] = ll[0].strip()
            except IndexError:
                pass
            
            
        if n not in n_val.keys():
            continue
        if n == 11:
            try:
                int(tds[-1].text.strip())
            except ValueError:
                continue
        temp[n_val[n]] = tds[-1].text
        

    temp["name"] = sp.select("#_listedcompanyportlet_WAR_HelexServiceportlet_companyname")[0].text
    temp["stockexchanges"] = ["AT"]
    temp["listed_on"] = {"AT":temp["listed_on"]}

    

    with open(os.path.join(p, ticker+".json"), "w") as f:
        json.dump(temp, f)

https://www.athexgroup.gr/company-profile/-/select-company/1173
[<td class="column-width-25 left-col">Sector / Subsector</td>, <td class="column-width-75 right-col"> </td>]
https://www.athexgroup.gr/company-profile/-/select-company/1515
[<td class="column-width-25 left-col">Sector / Subsector</td>, <td class="column-width-75 right-col"> Industrial Goods &amp; Services / Transportation Services (Feb 7, 2024) </td>]
https://www.athexgroup.gr/company-profile/-/select-company/791
[<td class="column-width-25 left-col">Sector / Subsector</td>, <td class="column-width-75 right-col"> Construction &amp; Materials / Building Materials: Other (Jul 1, 2019) </td>]
https://www.athexgroup.gr/company-profile/-/select-company/555
[<td class="column-width-25 left-col">Sector / Subsector</td>, <td class="column-width-75 right-col"> Basic Resources / Aluminum (Jul 1, 2019) </td>]
https://www.athexgroup.gr/company-profile/-/select-company/278
[<td class="column-width-25 left-col">Sector / Subsector</td>, 

AttributeError: 'NoneType' object has no attribute 'group'

In [2]:
url = "https://www.athexgroup.gr/web/guest/stock-company-profile/-/select-stock/921"
resp = requests.get(url)
soup = BeautifulSoup(resp.text)

NameError: name 'requests' is not defined

In [ ]:
import re
m = re.match(r"\s", " ")
m

False

In [ ]:
import yfinance as yf, json, os
p = "/Users/a2/code/fin/trade/data/fundamentals/company/info"
for file in os.listdir(p):
    with open(os.path.join(p, file)) as f:
        temp = json.load(f)
    t = file.replace("json", "AT")
    temp["tickers"] = {"AT": t}
    tt = yf.Ticker(t)
    info = tt.info
    temp['activity'] = info['longBusinessSummary']
    